In [3]:
# !pip install tensorboard
# !pip install pytorch-lightning
# !pip install torch-geometric

In [1]:
import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from pygcn.utils import load_data, accuracy
from pygcn.models import GCN

from torch.utils.tensorboard import SummaryWriter

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

from torch_geometric.datasets import Planetoid

In [2]:
seed = 42

np.random.seed(seed)
torch.manual_seed(seed)
torch.mps.manual_seed(seed)

In [3]:
class CORA_Dataset(torch.utils.data.Dataset):
    def __init__(self):
        super(CORA_Dataset).__init__()
        self.data = Planetoid(root='./cora/', name='cora')
        
    def __len__(self):
        return 1
    
    def __getitem__(self, idx):
        return self.data[0]
    
dataset = CORA_Dataset()
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

In [4]:
# class CORA_Dataset(torch.utils.data.Dataset):
#     def __init__(self, adj, features, labels, idx_train, idx_val, idx_test):
#         super(CORA_Dataset).__init__()
#         self.adj = adj
#         self.features = features
#         self.labels = labels
#         self.idx_train = idx_train
#         self.idx_val = idx_val
#         self.idx_test = idx_test
        
#     def __len__(self):
#         return 1
    
#     def __getitem__(self, idx):
#         return self.adj, self.features, self.labels, self.idx_train, self.idx_val, self.idx_test
    
# dataset = CORA_Dataset(*load_data('pygcn/data/cora/'))
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

In [5]:
nfeat = dataset.data.x.size()[1]
hidden = 16
nclass = dataset.data.y.max().item() + 1
dropout = 0.5
laplac_size = dataset.data.edge_index.size(1)

weight_decay = 5e-4

In [6]:
class GCN_module(pl.LightningModule):
    def __init__(self, learning_rate=0.01):
        super(GCN_module, self).__init__()
        self.model = GCN(nfeat=nfeat,
                         nhid=hidden,
                         nclass=nclass,
                         dropout=dropout,
                         laplac_size=laplac_size)
        
        self.learning_rate = learning_rate
        self.loss_fn = F.nll_loss
        
    def configure_optimizers(self):
        optim = torch.optim.Adam(self.parameters(), lr=self.learning_rate,
                           weight_decay=weight_decay)
        return optim
    
    def forward(self, features, adj):
        return self.model(features, adj)
    
    
    def training_step(self, train_batch, batch_idx):
        data = train_batch
        output = self.model(data.x, data.edge_index)
        
        labels = data.y
        
        loss_train = self.loss_fn(output[data.train_mask], labels[data.train_mask])
        acc_train = accuracy(output[data.train_mask], labels[data.train_mask])
        
        loss_val = self.loss_fn(output[data.val_mask], labels[data.val_mask])
        acc_val = accuracy(output[data.val_mask], labels[data.val_mask])
        
        self.log("loss_train", loss_train, prog_bar=True)
        self.log("acc_train", acc_train, prog_bar=True)
        
        self.log("loss_val", loss_val, prog_bar=True)
        self.log("acc_val", acc_val, prog_bar=True)
        
        return loss_train
        
    def validation_step(self, val_batch, batch_idx):
        pass
        

In [7]:
# !rm -r lightning_logs

In [8]:
# checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="acc_val", mode="max")

device = "cpu"
trainer = pl.Trainer(max_epochs=400, accelerator=device)#, callbacks=checkpoint_callback, )

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/i.betev/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


In [ ]:
module = GCN_module()

trainer.fit(module, dataloader, dataloader)

In [38]:
!rm -r runs

In [30]:
writer = SummaryWriter()

In [31]:
adj, features, labels, idx_train, idx_val, idx_test = load_data('pygcn/data/cora/')

Loading cora dataset...


In [32]:
hidden = 16
dropout = 0.5
lr = 0.01
weight_decay = 5e-4
device = "cpu"

model = GCN(nfeat=features.shape[1],
            nhid=hidden,
            nclass=labels.max().item() + 1,
            dropout=dropout)

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

model.to(device)
features = features.to(device)
adj = adj.to(device)
labels = labels.to(device)
idx_train = idx_train.to(device)
idx_val = idx_val.to(device)
idx_test = idx_test.to(device)


In [33]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    # log
    # writer.add_scalar("Loss/train", loss_train, epoch)
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    # if not args.fastmode:
    #     # Evaluate validation set performance separately,
    #     # deactivates dropout during validation run.
    #     model.eval()
    #     output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))
    
    
    
    

def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))


In [34]:
epochs = 200

t_total = time.time()
for epoch in range(epochs):
    train(epoch)
    
writer.flush()


print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

Epoch: 0001 loss_train: 2.0034 acc_train: 0.1214 loss_val: 2.0204 acc_val: 0.1133 time: 0.0047s
Epoch: 0002 loss_train: 1.9814 acc_train: 0.1214 loss_val: 2.0026 acc_val: 0.1133 time: 0.0029s
Epoch: 0003 loss_train: 1.9625 acc_train: 0.1429 loss_val: 1.9839 acc_val: 0.1200 time: 0.0032s
Epoch: 0004 loss_train: 1.9549 acc_train: 0.1857 loss_val: 1.9724 acc_val: 0.1333 time: 0.0030s
Epoch: 0005 loss_train: 1.9371 acc_train: 0.2357 loss_val: 1.9581 acc_val: 0.1533 time: 0.0029s
Epoch: 0006 loss_train: 1.9290 acc_train: 0.2357 loss_val: 1.9486 acc_val: 0.1667 time: 0.0031s
Epoch: 0007 loss_train: 1.9130 acc_train: 0.2357 loss_val: 1.9350 acc_val: 0.1633 time: 0.0029s
Epoch: 0008 loss_train: 1.9043 acc_train: 0.2286 loss_val: 1.9261 acc_val: 0.1633 time: 0.0030s
Epoch: 0009 loss_train: 1.8993 acc_train: 0.2286 loss_val: 1.9165 acc_val: 0.1600 time: 0.0026s
Epoch: 0010 loss_train: 1.8824 acc_train: 0.2286 loss_val: 1.9033 acc_val: 0.1700 time: 0.0027s
Epoch: 0011 loss_train: 1.8801 acc_train

In [35]:
writer.close()

In [36]:
# tensorboard --logdir=runs

In [37]:
# Testing
test()

Test set results: loss= 0.7108 accuracy= 0.8290
